<a href="https://colab.research.google.com/github/so-dipe/IBM-DS-PC/blob/main/Capstone/Interactive%20DashBoard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2022-09-23 19:33:01--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.42K  --.-KB/s    in 0s      

2022-09-23 19:33:02 (487 MB/s) - ‘spacex_launch_dash.csv’ saved [2476/2476]



In [ ]:
! wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"

--2022-09-23 19:33:24--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2110 (2.1K) [text/x-python]
Saving to: ‘spacex_dash_app.py’

spacex_dash_app.py  100%[===================>]   2.06K  --.-KB/s    in 0s      

2022-09-23 19:33:25 (475 MB/s) - ‘spacex_dash_app.py’ saved [2110/2110]



In [14]:
! pip install dash
!pip install jupyter-dash

In [15]:
from jupyter_dash import JupyterDash
import pandas as pd
import dash
from dash import html, dash, dcc
# import dash_html_components as html
# import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [68]:
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [28]:
launch_sites = list(spacex_df['Launch Site'].unique())
launch_sites.append('All Sites')

['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40', 'All Sites']

In [93]:
# app = dash.Dash(__name__)
app = JupyterDash(__name__)


# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)

def pie_chart(option):
  if option == 'All Sites':
    success_df = spacex_df[spacex_df['class'] == 1]
    fig = px.pie(
        data_frame=success_df,
        names='Launch Site',
        title='Total Success by Launch Sites'
        # values='class'
    )
  else:
    launch_df = spacex_df[spacex_df['Launch Site'] == option]
    fig = px.pie(
        data_frame=launch_df,
        names='class',
        title=f'Total Success Launchs for site {option}'
        # values='Flight Number'
    )
  return fig



# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    Input('site-dropdown', 'value'),
    Input('payload-slider', 'value')
)

def scatter(site, range):
  range_df = spacex_df[spacex_df['Payload Mass (kg)'].between(range[0], range[1])]
  if site == 'All Sites':
    fig = px.scatter(
        data_frame=range_df,
        x='Payload Mass (kg)',
        y='class',
        color="Booster Version Category",
        title='Correlation between Payload and Success for all Sites'
    )
  else:
    site_range_df = range_df[range_df['Launch Site'] == site]
    fig = px.scatter(
        data_frame=site_range_df,
        x='Payload Mass (kg)',
        y='class',
        color="Booster Version Category",
        title=f'Correlation between Payload and Success for site {site}'
    )
  return fig



app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown', options=launch_sites, value='All Sites'),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000,
                                                step=1000, value=[min_payload, max_payload]
                                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])


if __name__ == '__main__':
    app.run_server()

Dash app running on:


<IPython.core.display.Javascript object>

Dash app running on:


<IPython.core.display.Javascript object>